In [1]:
from head import opentrons
import pandas as pd
import numpy as np
from scipy.spatial import distance
import warnings
warnings.filterwarnings("ignore")

INFO: Using numpy backend


In [2]:
class Simulator:
    def __init__(self):
        self.domain = np.linspace(-5,5,num=100)
        
    def generate(self, mu, sig):
        scale = 1/(np.sqrt(2*np.pi)*sig)
        return scale*np.exp(-np.power(self.domain - mu, 2.) / (2 * np.power(sig, 2.)))
    
    def process_batch(self, Cb, fname):
        out = []
        for c in Cb:
            out.append(self.generate(*c))
        out = np.asarray(out)
        df = pd.DataFrame(out.T, index=self.domain)
        df.to_excel(fname, engine='openpyxl')
        
        return 
    
    def make_target(self, ct):
        return self.domain, self.generate(*ct)

In [3]:
sim = Simulator()
target = np.array([-2,0.5])
xt, yt = sim.make_target(target)

In [4]:
from head import AmplitudePhaseDistance

def APdist(f1,f2):
    da, dp = AmplitudePhaseDistance(f1,f2,xt)
    
    return -(da+dp)
    

In [5]:
Cmu = [-5,5]
Csig = [0.1,3.5]
bounds = [Cmu, Csig]

In [6]:
optim = opentrons.Optimizer(xt, yt, 
                            bounds, 
                            savedir = '../data',
                            batch_size=4,
                            metric = APdist
                           )


INFO: Getting suggestions for iteration 0
INFO: Iteration : 0, Samples to be made
INFO: 0	[-1.69592045  1.42987791]
INFO: 1	[1.85998103 0.5546689 ]
INFO: 2	[2.27575725 0.18992321]
INFO: 3	[1.47655087 0.11815148]
INFO: ==============================


In [7]:
# random iteration
optim.save()
C0 = np.load('../data/0/new_x.npy')
sim.process_batch(C0, '../data/0.xlsx')
optim.update('../data/0.xlsx')
optim.save()
optim.get_current_best()

INFO: Iteration : 1 Best estimate [[0.04859615 1.27705016]] with predicted score : [[-0.4875459]]


In [8]:
for i in range(1,21):
    # iteration i selection
    optim.suggest_next()
    optim.save()
    # simulate iteration i new_x 
    Ci = np.load('../data/%d/new_x.npy'%i)
    sim.process_batch(Ci, '../data/%d.xlsx'%i)
    optim.update('../data/%d.xlsx'%i)
    optim.save()
    optim.get_current_best()

INFO: Getting suggestions for iteration 1
INFO: Iteration : 1, Samples to be made
INFO: 0	[2.19137254 1.83956224]
INFO: 1	[-4.99        3.03395053]
INFO: 2	[-4.99        0.28347929]
INFO: 3	[5.  3.5]
INFO: ==============================
INFO: Iteration : 2 Best estimate [[-0.40621184  1.18994157]] with predicted score : [[-0.55240365]]
INFO: Getting suggestions for iteration 2
INFO: Iteration : 2, Samples to be made
INFO: 0	[-3.65465249  1.33292663]
INFO: 1	[5.        1.2580402]
INFO: 2	[-1.55939771  0.84231223]
INFO: 3	[0.20746676 1.2234658 ]
INFO: ==============================
INFO: Iteration : 3 Best estimate [[-1.3762159   0.92881037]] with predicted score : [[-0.42388524]]
INFO: Getting suggestions for iteration 3
INFO: Iteration : 3, Samples to be made
INFO: 0	[-3.68951548  0.97710218]
INFO: 1	[-1.62155807  0.47493523]
INFO: 2	[-0.68822277  0.86495815]
INFO: 3	[-1.98945986  0.81463206]
INFO: ==============================
INFO: Iteration : 4 Best estimate [[-1.04998468  0.879403